# DNN Critic model

In [3]:
import pandas as pd 
import numpy as np  
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# read data
mydt = pd.read_csv("../dataset/UGR/trainspam.csv")
print(mydt['Label'].value_counts())
# Create matrix of features and matrix of target variable 
dataset = mydt.values
X = dataset[:,0:19]
y = dataset[:,19]

# convert the scaled array to dataframe 
#min_max_scaler = MinMaxScaler()
#X_scale = min_max_scaler.fit_transform(X)

X_full = X
y_full = y

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=1)

print(Counter(y_train))
print(Counter(y_test))

# define the keras model
model = Sequential()
model.add(Dense(128, input_shape=(19,), activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Building Critic WGAN model...")
# fit the keras model on the dataset
mod = model.fit(X_train, y_train, epochs=100, batch_size=30, verbose=1, validation_data=(X_test, y_test))

# make class predictions with the model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100))

%matplotlib inline
# Testing 
cm = confusion_matrix(y_test, y_pred) 
tnr = recall_score(y_test, y_pred) 
fpr = 1 - tnr

print ("confusion_matrix for test data\n",cm)
print(classification_report(y_test,y_pred,labels=np.unique(y_pred)))
print('Accuracy: %.4f' % accuracy_score(y_test, y_pred))
print('Precision: %.4f' % precision_score(y_test, y_pred))
print('Recall: %.4f' % recall_score(y_test, y_pred))
print('F1 Score: %.4f' % f1_score(y_test, y_pred))
print('FPR Score: %.4f' % fpr)

# Save model
# serialize model to JSON
model_json = model.to_json()
with open("DNN_critic_model_UGR_3.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("DNN_critic_model_UGR_3.h5")
print("Saved model!")

0    14029
1      266
Name: Label, dtype: int64
Counter({0.0: 9804, 1.0: 202})
Counter({0.0: 4225, 1.0: 64})
Building Critic WGAN model...
Epoch 1/100
334/334 [==============================] - 3s 6ms/step - loss: 0.1186 - accuracy: 0.9734 - val_loss: 0.0576 - val_accuracy: 0.9851
Epoch 2/100
334/334 [==============================] - 2s 5ms/step - loss: 0.0545 - accuracy: 0.9798 - val_loss: 0.0356 - val_accuracy: 0.9851
Epoch 3/100
334/334 [==============================] - 2s 5ms/step - loss: 0.0317 - accuracy: 0.9844 - val_loss: 0.0249 - val_accuracy: 0.9876
Epoch 4/100
334/334 [==============================] - 1s 4ms/step - loss: 0.0253 - accuracy: 0.9894 - val_loss: 0.0222 - val_accuracy: 0.9904
Epoch 5/100
334/334 [==============================] - 2s 5ms/step - loss: 0.0228 - accuracy: 0.9904 - val_loss: 0.0218 - val_accuracy: 0.9902
Epoch 6/100
334/334 [==============================] - 1s 4ms/step - loss: 0.0224 - accuracy: 0.9907 - val_loss: 0.0216 - val_accuracy: 0.9902
Epo

Epoch 57/100
334/334 [==============================] - 1s 4ms/step - loss: 0.0116 - accuracy: 0.9953 - val_loss: 0.0229 - val_accuracy: 0.9923
Epoch 58/100
334/334 [==============================] - 1s 4ms/step - loss: 0.0110 - accuracy: 0.9957 - val_loss: 0.0228 - val_accuracy: 0.9925
Epoch 59/100
334/334 [==============================] - 1s 3ms/step - loss: 0.0108 - accuracy: 0.9959 - val_loss: 0.0240 - val_accuracy: 0.9925
Epoch 60/100
334/334 [==============================] - 1s 4ms/step - loss: 0.0112 - accuracy: 0.9955 - val_loss: 0.0236 - val_accuracy: 0.9923
Epoch 61/100
334/334 [==============================] - 1s 3ms/step - loss: 0.0101 - accuracy: 0.9963 - val_loss: 0.0267 - val_accuracy: 0.9904
Epoch 62/100
334/334 [==============================] - 1s 3ms/step - loss: 0.0101 - accuracy: 0.9959 - val_loss: 0.0247 - val_accuracy: 0.9921
Epoch 63/100
334/334 [==============================] - 1s 3ms/step - loss: 0.0100 - accuracy: 0.9959 - val_loss: 0.0270 - val_accuracy:

# WGAN

In [4]:
import torch 
import torch.nn
import torch.nn.functional as nn 
import torch.autograd as autograd 
import torch.optim as optim 
import torch.utils.data as Data
from torch.autograd import Variable
import numpy as np 
import tensorflow as tf 
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import confusion_matrix 
import keras
from keras.models import Sequential 
from keras.layers import Dense 
from keras import optimizers
from keras.models import model_from_json
import matplotlib 
matplotlib.use('agg') 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 

# hyperparameters 
mb_size = 64 
z_dim = 10 
X_dim = 19 
y_dim = 1 
h_dim = 128 #hidden 
lr = 0.00005 #learning_rate

# read data
df = pd.read_csv("../dataset/UGR/trainspam.csv")
print('original data label features number:')
print(df['Label'].value_counts())

# data preparation
criteria0 = df['Label'] == 1 #filter attack data 
attack_data = data0 = df[criteria0]

#float_array = data0.values.astype(float) 
#scaler = MinMaxScaler()
#scaled_array = scaler.fit_transform(float_array)
#attack_data = pd.DataFrame(scaled_array)
#attack_data.columns = ['hour','minute','second','duration','source_ip','dest_ip','source_port','dest_port','protocol',
 #               'flag1','flag2','flag3','flag4','flag5','flag6','fwd','stos','pkt','byt', 'Label']

print('----------------------------------------') 
print('attack value number:') 
print(attack_data['Label'].value_counts())

# transform data to tensor format
train_target = torch.tensor(attack_data['Label'].values.astype(np.float32)) 
train = torch.tensor(attack_data.drop(['Label'], axis = 1).values.astype(np.float32)) 
train_tensor = Data.TensorDataset(train, train_target) 

#creat DataLoader with batch and shuffle
train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = mb_size, shuffle = True)

# define models
# Generator
G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim), 
    torch.nn.Sigmoid()
)

# Discriminator
D = torch.nn.Sequential( 
    torch.nn.Linear(X_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim), 
    torch.nn.ReLU(),
    # No sigmoid
    torch.nn.Linear(h_dim, 1),
)

# setup cpu using cpu (if use gpu -> set cpu to cuda)
G.cpu()
D.cpu()

def reset_grad():
    G.zero_grad()
    D.zero_grad()
    
#initialize optimizers and dataholders
G_solver = optim.RMSprop(G.parameters(), lr=lr)
D_solver = optim.RMSprop(D.parameters(), lr=lr)
G_losses = []
D_losses = []
data_iter = iter(train_loader)

original data label features number:
0    14029
1      266
Name: Label, dtype: int64
----------------------------------------
attack value number:
1    266
Name: Label, dtype: int64


In [5]:
#training 
print("Start training WGAN model...")
d_iter = 2 
for it in range(500000):
    if it<100: d_iter = 50 
    else:
        d_iter = 2
    for _ in range(d_iter):
        # Sample data
        z = Variable(torch.randn(mb_size, z_dim)).cpu()
        try:
            X, _ = data_iter.next() 
        except StopIteration:
            data_iter = iter(train_loader) 
            X, _ = data_iter.next()
        #X = Variable(torch.from_numpy(X))
        X = X.cpu()
        
        # Dicriminator forward-loss-backward-update 
        G_sample = G(z)
        D_real = D(X)
        D_fake = D(G_sample)
        
        D_loss = -(torch.mean(D_real) - torch.mean(D_fake))
        
        D_loss.backward() 
        D_solver.step()
        
        # Weight clipping 
        for p in D.parameters():
            p.data.clamp_(-0.01, 0.01)
            
        D_losses.append(D_loss.data.cpu().numpy())
        
        # Housekeeping - reset gradient 
        reset_grad()
        
    # Generator forward-loss-backward-update 
    #X = Variable(torch.from_numpy(X)) 
    z = Variable(torch.randn(mb_size, z_dim)).cpu() #change cpu to cuda
    
    G_sample = G(z)
    D_fake = D(G_sample)
    
    G_loss = -torch.mean(D_fake)
    
    G_loss.backward() 
    G_solver.step()

    # Housekeeping - reset gradient 
    reset_grad()

    G_losses.append(G_loss.data.cpu().numpy())

    # Print and plot every now and then 
    if it % 1000 == 0:
        print('Iter-{}; D_loss: {}; G_loss: {}'
              .format(it, D_loss.data.cpu().numpy(),  
                G_loss.data.cpu().numpy()))
        
        samples = G(z).data.numpy()[:16]

#plot losses
plt.figure(figsize=(10,5))
plt.title("Generator and Critic Loss During Training") 
plt.plot(G_losses,label="Generator") 
plt.plot(D_losses,label="Critic") 
plt.xlabel("iter")
plt.ylabel("loss")
plt.legend()
plt.savefig('UGR_loss_3.png')

#save model
torch.save({'modelG_state_dict': G.state_dict(), 
            'modelD_state_dict': D.state_dict(),
            'optimizerG_state_dict': G_solver.state_dict(),
            'optimizerD_state_dict': D_solver.state_dict(),
            'G_loss': G_losses,
            'D_loss': D_losses
           }, "WGAN_model_UGR_3.pth")

Start training WGAN model...
Iter-0; D_loss: -0.0026344265788793564; G_loss: -0.008851490914821625
Iter-1000; D_loss: -0.00012710876762866974; G_loss: -0.013564711436629295
Iter-2000; D_loss: -1.7745420336723328e-05; G_loss: -0.013688127510249615
Iter-3000; D_loss: -7.441267371177673e-07; G_loss: -0.009884634986519814
Iter-4000; D_loss: -0.0030577797442674637; G_loss: -0.015626441687345505
Iter-5000; D_loss: -0.0013016723096370697; G_loss: -0.011436956003308296
Iter-6000; D_loss: -0.000163322314620018; G_loss: -0.013660922646522522
Iter-7000; D_loss: -0.00048270542174577713; G_loss: -0.013561341911554337
Iter-8000; D_loss: -8.635502308607101e-05; G_loss: -0.013333861716091633
Iter-9000; D_loss: -0.00017167441546916962; G_loss: -0.013631363399326801
Iter-10000; D_loss: 0.00013803225010633469; G_loss: -0.014054855331778526
Iter-11000; D_loss: -0.0003743758425116539; G_loss: -0.013823937624692917
Iter-12000; D_loss: -0.00011842604726552963; G_loss: -0.01403164304792881
Iter-13000; D_loss:

Iter-111000; D_loss: 6.0187652707099915e-05; G_loss: -0.013178137131035328
Iter-112000; D_loss: 9.485706686973572e-05; G_loss: -0.01224270835518837
Iter-113000; D_loss: -0.0004893019795417786; G_loss: -0.01323945727199316
Iter-114000; D_loss: 1.0478310286998749e-05; G_loss: -0.013685495592653751
Iter-115000; D_loss: 3.820285201072693e-05; G_loss: -0.01323041319847107
Iter-116000; D_loss: 0.00014708098024129868; G_loss: -0.013074754737317562
Iter-117000; D_loss: -0.0002483539283275604; G_loss: -0.013178451918065548
Iter-118000; D_loss: 0.00033898185938596725; G_loss: -0.013881008140742779
Iter-119000; D_loss: 0.0004097782075405121; G_loss: -0.01291400846093893
Iter-120000; D_loss: 9.795650839805603e-05; G_loss: -0.012336527928709984
Iter-121000; D_loss: -0.0003776922821998596; G_loss: -0.01346430554986
Iter-122000; D_loss: -0.0002663480117917061; G_loss: -0.013501809909939766
Iter-123000; D_loss: -0.00012369081377983093; G_loss: -0.013385982252657413
Iter-124000; D_loss: -2.159737050533

Iter-221000; D_loss: -6.823800504207611e-06; G_loss: -0.013857719488441944
Iter-222000; D_loss: -6.0051679611206055e-06; G_loss: -0.013860220089554787
Iter-223000; D_loss: -2.4978071451187134e-06; G_loss: -0.014066597446799278
Iter-224000; D_loss: 4.091300070285797e-06; G_loss: -0.013916289433836937
Iter-225000; D_loss: 1.1214055120944977e-05; G_loss: -0.013936677947640419
Iter-226000; D_loss: -1.9609928131103516e-05; G_loss: -0.01411453541368246
Iter-227000; D_loss: 1.6744248569011688e-05; G_loss: -0.013913869857788086
Iter-228000; D_loss: 8.940696716308594e-08; G_loss: -0.01389954425394535
Iter-229000; D_loss: 1.6968697309494019e-06; G_loss: -0.014033171348273754
Iter-230000; D_loss: 4.453584551811218e-06; G_loss: -0.014037306420505047
Iter-231000; D_loss: -2.605840563774109e-06; G_loss: -0.014077886939048767
Iter-232000; D_loss: -8.236616849899292e-06; G_loss: -0.014150525443255901
Iter-233000; D_loss: -1.3072974979877472e-05; G_loss: -0.014051984995603561
Iter-234000; D_loss: 8.138

Iter-331000; D_loss: -1.1702068150043488e-05; G_loss: -0.013675695285201073
Iter-332000; D_loss: 3.9637088775634766e-05; G_loss: -0.013828619383275509
Iter-333000; D_loss: 9.834766387939453e-06; G_loss: -0.013576728291809559
Iter-334000; D_loss: 1.9060447812080383e-05; G_loss: -0.013898542150855064
Iter-335000; D_loss: -1.4967285096645355e-05; G_loss: -0.013551978394389153
Iter-336000; D_loss: 9.052455425262451e-07; G_loss: -0.013641643337905407
Iter-337000; D_loss: -1.5659257769584656e-05; G_loss: -0.01371755637228489
Iter-338000; D_loss: -1.1683441698551178e-05; G_loss: -0.013945288956165314
Iter-339000; D_loss: 2.0703300833702087e-05; G_loss: -0.013823158107697964
Iter-340000; D_loss: 6.410758942365646e-05; G_loss: -0.013753972947597504
Iter-341000; D_loss: -3.0133873224258423e-05; G_loss: -0.0140620032325387
Iter-342000; D_loss: 3.845337778329849e-05; G_loss: -0.013765930198132992
Iter-343000; D_loss: -4.79724258184433e-06; G_loss: -0.013833178207278252
Iter-344000; D_loss: 1.82725

Iter-441000; D_loss: 1.8505379557609558e-06; G_loss: -0.012530706822872162
Iter-442000; D_loss: -5.444511771202087e-06; G_loss: -0.012745020911097527
Iter-443000; D_loss: -4.152767360210419e-06; G_loss: -0.011942366138100624
Iter-444000; D_loss: -9.98377799987793e-07; G_loss: -0.011324294842779636
Iter-445000; D_loss: 3.975816071033478e-06; G_loss: -0.011255215853452682
Iter-446000; D_loss: -6.05359673500061e-08; G_loss: -0.010971155017614365
Iter-447000; D_loss: 1.1604279279708862e-06; G_loss: -0.012164182029664516
Iter-448000; D_loss: 2.1131709218025208e-06; G_loss: -0.012154822237789631
Iter-449000; D_loss: -3.563240170478821e-06; G_loss: -0.011905500665307045
Iter-450000; D_loss: 8.391216397285461e-07; G_loss: -0.011743037961423397
Iter-451000; D_loss: -6.862916052341461e-06; G_loss: -0.011956136673688889
Iter-452000; D_loss: -7.022172212600708e-06; G_loss: -0.011884837411344051
Iter-453000; D_loss: 1.7723068594932556e-06; G_loss: -0.0121703390032053
Iter-454000; D_loss: 7.16652721

# Data generation from saved WGAN model

In [6]:
#loaded NN model
print("Loading model...")
print("----------------------------------------")
json_file = open("DNN_critic_model_UGR_3.json","r") 
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json) 
loaded_model.load_weights("DNN_critic_model_UGR_3.h5")

G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim), 
    torch.nn.Sigmoid()
)

checkpoint = torch.load("WGAN_model_UGR_3.pth")
G.load_state_dict(checkpoint['modelG_state_dict'])

G.eval()
G.cpu()

final_Data = pd.DataFrame(columns = ['hour','minute','second','duration','source_ip','dest_ip','source_port','dest_port',
                                     'protocol','flag1','flag2','flag3','flag4','flag5','flag6',
                                     'fwd','stos','pkt','byt','Label']) 
print("empty df",final_Data) 
print("----------------------------------------")
print("Completed loading model")

Loading model...
----------------------------------------
empty df Empty DataFrame
Columns: [hour, minute, second, duration, source_ip, dest_ip, source_port, dest_port, protocol, flag1, flag2, flag3, flag4, flag5, flag6, fwd, stos, pkt, byt, Label]
Index: []
----------------------------------------
Completed loading model


In [7]:
it =0

while(final_Data.shape[0]<20000):
    it+=1
    z = Variable(torch.randn(100, z_dim)).cpu() 
    with torch.no_grad():
        data = G(z)
    data = data.cpu().numpy() 
    fake = pd.DataFrame(data)
        
    newy = np.ones((fake.shape[0],)) 
    #this is the model which changes 
    new_pred = loaded_model.predict(fake)
    new_pred = (new_pred > 0.5)
    cm = confusion_matrix(newy,new_pred)
    print("-------------------------------------------------------------") 
    print (" confusion_matrix for new data with deeper model \n" ,cm)
    
    combined_data = np.concatenate((fake,new_pred),axis=1) 
    combined_data = pd.DataFrame(combined_data)
    combined_data.columns = ['hour','minute','second','duration','source_ip','dest_ip','source_port','dest_port','protocol',
                'flag1','flag2','flag3','flag4','flag5','flag6','fwd','stos','pkt','byt', 'Label']
    
    criteria1 = combined_data['Label'] == 1 #just attack data 
    data1 = combined_data[criteria1]
    print("only attack data - ",data1.shape) 
    final_Data = final_Data.append(data1, ignore_index = True) 
    print("final data ", it, "- ",final_Data.shape) 
    print("-------------------------------------------------------------") 

print(final_Data['Label'].value_counts())
print("---------------------------------------------------------")
df2 = final_Data.sample(n=13763, replace=False, random_state=1)
df3 = final_Data.sample(n=20000, replace=False, random_state=1)
final = pd.concat([mydt,df2], ignore_index = True,axis=0)
final['Label'].value_counts()
df3.to_csv("../dataset/UGR/generated_spam.csv", index = False, header = True)
final.to_csv("../dataset/UGR/balanced_data_spam.csv", index = False, header = True)

4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [22 78]]
only attack data -  (78, 20)
final data  1 -  (78, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [22 78]]
only attack data -  (78, 20)
final data  2 -  (156, 20)
-------------------------------------------------------------


/tmp/ipykernel_743181/4222328789.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_743181/4222328789.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [30 70]]
only attack data -  (70, 20)
final data  3 -  (226, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [26 74]]
only attack data -  (74, 20)
final data  4 -  (300, 20)
-------------------------------------------------------------


/tmp/ipykernel_743181/4222328789.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_743181/4222328789.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [29 71]]
only attack data -  (71, 20)
final data  5 -  (371, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [33 67]]
only attack data -  (67, 20)
final data  6 -  (438, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [25 75]]
only attack data -  (75, 20)
final data  7 -  (513, 20)
-------------------------------------------------------------


...

/tmp/ipykernel_743181/4222328789.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_743181/4222328789.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_743181/4222328789.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [24 76]]
only attack data -  (76, 20)
final data  270 -  (19907, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [32 68]]
only attack data -  (68, 20)
final data  271 -  (19975, 20)
-------------------------------------------------------------


/tmp/ipykernel_743181/4222328789.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_743181/4222328789.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [31 69]]
only attack data -  (69, 20)
final data  272 -  (20044, 20)
-------------------------------------------------------------
1.0    20044
Name: Label, dtype: int64
---------------------------------------------------------


/tmp/ipykernel_743181/4222328789.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
